In [76]:
import pandas as pd
import time
import os
from selenium import webdriver

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [77]:
linkedin_url_1 = "https://www.linkedin.com/jobs/search/?currentJobId=3477276853&geoId=106677219&keywords=data%20analyst&location=Waterloo%2C%20Ontario%2C%20Canada&refresh=true"


In [78]:
# connect with the chrome driver
driver = webdriver.Chrome(executable_path=r"./chromedriver")

In [79]:
driver.get(linkedin_url_1)
driver.implicitly_wait(10)

现在在chrome里面已经load出来linkedin url了，下一步就是看有多少个job post了, 你根据这个job post的数字, 右键打开页面source code, 可以搜出对应的`class name` as `"results-context-header__job-count"`

In [80]:
# find element by the class name
n = driver.find_element_by_class_name("results-context-header__job-count").text

This is the number of job count in total we need to find

In [81]:
# convert string to numeric
y = pd.to_numeric(n)
y

366

但实际上，还有更多的工作并没有被没办法load出来，当你scroll down到bottom的时候，会sequentially发生以下两件事情:
- automatically loading new pages for like 3-4 times
- 当page上出现的job posting hits a limit, 会prompt you `see more jobs`,

所以我们现在要做的就是use `selenium` to keep scroll down

In [82]:
# i dummy iterator, the program time is actually controled by the time.sleep()
i = 0
while i <= 25:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    i += 1

    try:
        # find th button
        button_see_jobs = driver.find_element_by_class_name("infinite-scroller__show-more-button")

        
        # click it
        driver.execute_script("arguments[0].click();",button_see_jobs)
        
        #3 secs grace period for it to load
        time.sleep(2)
    except:
        # not finding the button, wait for three secs and continue
        pass
        time.sleep(1)
        

Now let's find the company name of the `Sunlife`

In [83]:
# set up two array for job title and company name, etc
company_name = []
job_title = []

for j in range(y):
    try:
        company = driver.find_elements_by_class_name('base-search-card__subtitle')[j].text
        job = driver.find_elements_by_class_name('base-search-card__title')[j].text
        
        company_name.append(company)
        job_title.append(job)
        
    except IndexError:
        print("done")

In [84]:
jd_link = []
findlink = driver.find_elements_by_class_name('base-card__full-link')

for k in findlink:
    jd_link.append(k.get_attribute('href'))

jd_link

['https://ca.linkedin.com/jobs/view/marketing-data-analyst-at-fusion-homes-3456506533?refId=aZN75DTNgQkPdJMR6IdbxA%3D%3D&trackingId=uAJnUi%2FK97jPRWX8ojZDxQ%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://ca.linkedin.com/jobs/view/lead-data-analyst-at-system1-3429798324?refId=aZN75DTNgQkPdJMR6IdbxA%3D%3D&trackingId=Em1f6HEdoqjn3NKZ%2FdACmg%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://ca.linkedin.com/jobs/view/data-production-analyst-at-sun-life-3462460706?refId=aZN75DTNgQkPdJMR6IdbxA%3D%3D&trackingId=lS5ElVyK9p5C2TyShcyk0Q%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://ca.linkedin.com/jobs/view/database-analyst-at-grand-river-hospital-3446202963?refId=aZN75DTNgQkPdJMR6IdbxA%3D%3D&trackingId=4riAP%2BINJ8Vs4GccV4er1Q%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'https://ca.linkedin.com/jobs/view/information-data-management-analyst-at-wilfrid-laurier-university-344

In [95]:
df = pd.DataFrame()
df["company"] = company_name
df["job_title"] = job_title
df["link"] = jd_link


In [96]:
df

,company,job_title,link
0,Fusion Homes,Marketing Data Analyst,https://ca.linkedin.com/jobs/view/marketing-da...
1,System1,Lead Data Analyst,https://ca.linkedin.com/jobs/view/lead-data-an...
2,Sun Life,Data Production Analyst,https://ca.linkedin.com/jobs/view/data-product...
3,Grand River Hospital,Database Analyst,https://ca.linkedin.com/jobs/view/database-ana...
4,Wilfrid Laurier University,Information/Data Management Analyst,https://ca.linkedin.com/jobs/view/information-...
...,...,...,...
361,Affirm,Principal Software Engineer (Developer Platform),https://ca.linkedin.com/jobs/view/principal-so...
362,Affinity,Sr. Software Engineer Node.js - Contract,https://ca.linkedin.com/jobs/view/sr-software-...
363,Canonical,Senior Software Engineer - Go,https://ca.linkedin.com/jobs/view/senior-softw...
364,EPAM Systems,Lead Java Developer,https://ca.linkedin.com/jobs/view/lead-java-de...
